In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from Methods.costs import * 
from Methods.least_squares import * 
from Methods.ridge import *
from Methods.cross_validation import *
from Methods.split_data import *
from Methods.scaling_standardization import *
from Methods.build_polynomial import *
from Methods.clearDataset import *

%load_ext autoreload
%autoreload 2

## Load the training data into feature matrix, class labels, and event ids:

In [2]:
from Methods.proj1_helpers import *
DATA_TRAIN_PATH = 'csv/train.csv'
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

## Generate predictions and save ouput in csv format for submission:

In [3]:
DATA_TEST_PATH = 'csv/test.csv' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

## Do your thing crazy machine learning thing here :) ...

# Feature Processing

In [4]:
tX, tX_test = averageData(tX, tX_test)
tX, tX_test = data_scaling(tX.T, tX_test.T)

## Find Best Model

In [5]:
#y, x, tX, method,  tX_test, x_test, **kwargs
for i in range(0, 30): 
    tX, tX_test = add_feature(y, tX[i], tX, log_def, tX_test, tX_test[i])
    tX, tX_test = add_feature(y, tX[i], tX, multiply, tX_test, tX_test[i], degree=2)
    tX, tX_test = add_feature(y, tX[i], tX, multiply, tX_test, tX_test[i], degree=3)
    tX, tX_test = add_feature(y, tX[i], tX, sqrt_def, tX_test, tX_test[i])

tX = tX.T

seed = 1
k_fold = 10

# split data in k fold
k_indices = build_k_indices(y, k_fold, seed)

# define lists to store the loss of training data and test data
rmse_tr = []
rmse_te = []
#k_list = list(range(k_fold))
k=7
tot_loss_tr = 0
tot_loss_te = 0
best_accuracy = 0
best_k = 0
weights = np.array([])
#for k in k_list:
loss_tr, loss_te, accuracy_least, w = cross_validation(y, tX, k_indices, k, least_squares)
tot_loss_tr += loss_tr
tot_loss_te += loss_te
if accuracy_least > best_accuracy:
    best_accuracy = accuracy_least
    best_k = k
    weights = w
rmse_tr.append(np.sqrt(2/k_fold * tot_loss_tr))
rmse_te.append(np.sqrt(2/k_fold * tot_loss_te))
print(best_accuracy)
print(best_k)
#print(weights)


/Users/JulieDjeffal/Documents/EPFL/pattern/2016/ML_project1/projects/project1/scripts/Methods/build_polynomial.py:29: RuntimeWarning: divide by zero encountered in log
  new = np.array([np.log(x)])
/Users/JulieDjeffal/Documents/EPFL/pattern/2016/ML_project1/projects/project1/scripts/Methods/build_polynomial.py:29: RuntimeWarning: invalid value encountered in log
  new = np.array([np.log(x)])
/Users/JulieDjeffal/anaconda/lib/python3.5/site-packages/numpy/lib/function_base.py:964: RuntimeWarning: invalid value encountered in multiply
  scl = np.multiply(avg, 0) + scl
/Users/JulieDjeffal/anaconda/lib/python3.5/site-packages/numpy/lib/function_base.py:2490: RuntimeWarning: invalid value encountered in subtract
  X -= avg[:, None]
/Users/JulieDjeffal/Documents/EPFL/pattern/2016/ML_project1/projects/project1/scripts/Methods/build_polynomial.py:25: RuntimeWarning: invalid value encountered in sqrt
  new = np.array([np.sqrt(x)])


[-1.95808836  0.13548839 -0.69782112 ...,  0.20715263 -0.80155664
 -0.86394769]
[-1.  1. -1. ...,  1. -1. -1.]
[-1. -1. -1. ...,  1. -1. -1.]
0.80712
7


### Binary Classification

In [7]:
OUTPUT_PATH = 'csv/sample-submission.csv' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(weights, tX_test.T)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)